In [1]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn import metrics

In [2]:
# dealing with data in sparse format
data = list()
ilist = list()
jlist = list()
with open('train.dat', 'r') as fr:
    for lineno, line in enumerate(fr):
        line = line.strip()
        #print(line)
        parts = line.split()
        #print(parts)
        for i in range(int(len(parts)/2)):
            # 1237 1 1390 1 1391 5 ...
            jlist.append(int(parts[i*2]))
            data.append(int(parts[i*2+1]))
            ilist.append(lineno)

cm = csr_matrix((data, (ilist, jlist)), dtype = np.float)
print(cm)
cm.toarray().shape

  (0, 4)	3.0
  (0, 6)	1.0
  (0, 20)	1.0
  (0, 22)	1.0
  (0, 30)	5.0
  (0, 35)	1.0
  (0, 41)	1.0
  (0, 46)	1.0
  (0, 49)	1.0
  (0, 61)	1.0
  (0, 69)	1.0
  (0, 78)	1.0
  (0, 85)	1.0
  (0, 105)	1.0
  (0, 117)	2.0
  (0, 121)	1.0
  (0, 135)	1.0
  (0, 142)	1.0
  (0, 145)	2.0
  (0, 169)	4.0
  (0, 175)	3.0
  (0, 205)	1.0
  (0, 206)	1.0
  (0, 210)	1.0
  (0, 262)	3.0
  :	:
  (8579, 7908)	1.0
  (8579, 8138)	1.0
  (8579, 8142)	1.0
  (8579, 8256)	1.0
  (8579, 10117)	1.0
  (8579, 10195)	1.0
  (8579, 10797)	1.0
  (8579, 11606)	1.0
  (8579, 11747)	1.0
  (8579, 12468)	1.0
  (8579, 12537)	1.0
  (8579, 12901)	1.0
  (8579, 13113)	2.0
  (8579, 13189)	1.0
  (8579, 13719)	1.0
  (8579, 13938)	1.0
  (8579, 13939)	1.0
  (8579, 19752)	1.0
  (8579, 21044)	1.0
  (8579, 22592)	1.0
  (8579, 25101)	1.0
  (8579, 27641)	1.0
  (8579, 32303)	1.0
  (8579, 49115)	2.0
  (8579, 85241)	3.0


(8580, 126356)

In [3]:
X_initial = np.array(cm)
X_initial

array(<8580x126356 sparse matrix of type '<class 'numpy.float64'>'
	with 1107980 stored elements in Compressed Sparse Row format>,
      dtype=object)

In [4]:
# tf-idf scaling with normalization
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(norm=None, use_idf=True, smooth_idf=True)
#transformer = TfidfTransformer(use_idf=True, smooth_idf=False)
tfidf = transformer.fit_transform(cm)

In [5]:
'''from gensim.models import word2vec


# Set values for various parameters
feature_size = 150    # Word vector dimensionality  
window_context = 5          # Context window size                                                                                    
min_word_count = 1   # Minimum word count                        
sample = 1e-3   # Downsample setting for frequent words

w2v_model = word2vec.Word2Vec(X_initial, size=feature_size, 
            )'''



'from gensim.models import word2vec\n\n\n# Set values for various parameters\nfeature_size = 150    # Word vector dimensionality  \nwindow_context = 5          # Context window size                                                                                    \nmin_word_count = 1   # Minimum word count                        \nsample = 1e-3   # Downsample setting for frequent words\n\nw2v_model = word2vec.Word2Vec(X_initial, size=feature_size, \n            )'

In [6]:
tfidf.shape

(8580, 126356)

In [7]:
cmArr = tfidf.toarray()

In [8]:
%%time
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=150)
#svd.fit(cm.toarray())
#print(svd.explained_variance_ratio_.sum())

CPU times: user 16.8 ms, sys: 5.61 ms, total: 22.4 ms
Wall time: 21.2 ms


In [9]:
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
#normalizer = Normalizer(copy=False)
#lsa = make_pipeline(svd, normalizer)

In [10]:
%%time
X = svd.fit_transform(cmArr)

CPU times: user 2min 34s, sys: 29.7 s, total: 3min 3s
Wall time: 52.6 s


In [11]:
'''from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=150, max_iter=1, random_state=0)
X = lda.fit_transform(cmArr)'''



'from sklearn.decomposition import LatentDirichletAllocation\nlda = LatentDirichletAllocation(n_components=150, max_iter=1, random_state=0)\nX = lda.fit_transform(cmArr)'

In [12]:
X.shape

(8580, 150)

In [13]:
# variance calculation
explained_variance = svd.explained_variance_ratio_.sum()
print(explained_variance)

0.4218340023718873


In [14]:
#Create a set of np.random K centroids for each feature of the given dataset
def randCent(dataSet,K):
    n=np.shape(dataSet)[1]
    centroids=np.mat(np.zeros((K,n)))
    for j in range(n):
        minValue=min(dataSet[:,j])
        maxValue=max(dataSet[:,j])
        rangeValues=float(maxValue-minValue)
        #Make sure centroids stay within the range of data
        centroids[:,j]=minValue+rangeValues*np.random.rand(K,1)
    # np.matrix
    return centroids

In [15]:
# euclidean distance measure
def distanceMeasure(vecOne, vecTwo):
    return np.sqrt(np.sum(np.power(vecOne-vecTwo,2)))

In [16]:
# K means clustering method
def kMeans(dataSet,K,distMethods=distanceMeasure,createCent=randCent):
    m=np.shape(dataSet)[0]
    clusterAssment=np.mat(np.zeros((m,2)))
    # np.matrix
    centroids=createCent(dataSet,K)
    clusterChanged=True
    
    while clusterChanged:
        clusterChanged=False
        for i in range(m):
            minDist=np.inf; minIndex=-2
            for j in range(K):
                distJI=distMethods(centroids[j,:],dataSet[i,:])
                if distJI<minDist:
                    minDist=distJI;minIndex=j
            if clusterAssment[i,0] != minIndex:
                clusterChanged=True
            clusterAssment[i,:]=minIndex,minDist**2
        #update all the centroids by taking the np.mean value of relevant data
        for cent in range(K):
            ptsInClust=dataSet[np.nonzero(clusterAssment[:,0].A == cent)[0]]
            centroids[cent,:]=np.mean(ptsInClust,axis=0)
    # (np.matrix, np.matrix)
    return centroids,clusterAssment

In [17]:
#bisecting K-means method
def bisectingKMeans(dataSet,K,numIterations):
    m,n=dataSet.shape
    clusterInformation = np.mat(np.zeros((m,2)))
    centroidList=[]
    minSSE = np.inf
    
    #At the first place, regard the whole dataset as a cluster and find the best clusters
    for i in range(numIterations):
        # (np.matrix, np.matrix)
        centroid,clusterAssment=kMeans(dataSet, 2)
        SSE=np.sum(clusterAssment,axis=0)[0,1]
        if SSE<minSSE:
            minSSE=SSE
            tempCentroid=centroid
            tempCluster=clusterAssment
    centroidList.append(tempCentroid[0].tolist()[0])
    centroidList.append(tempCentroid[1].tolist()[0])
    clusterInformation=tempCluster
    minSSE=np.inf 
    
    while len(centroidList)<K:
        maxIndex=-2
        maxSSE=-1
        #Choose the cluster with Maximum SSE to split
        for j in range(len(centroidList)):
            SSE=np.sum(clusterInformation[np.nonzero(clusterInformation[:,0]==j)[0]])
            if SSE>maxSSE:
                maxIndex=j
                maxSSE=SSE
                
        #Choose the clusters with minimum total SSE to store into the centroidList
        for k in range(numIterations):
            pointsInCluster=dataSet[np.nonzero(clusterInformation[:,0]==maxIndex)[0]]
            # (np.matrix, np.matrix)
            centroid,clusterAssment=kMeans(pointsInCluster, 2)
            SSE=np.sum(clusterAssment[:,1],axis=0)
            if SSE<minSSE:
                minSSE=SSE
                tempCentroid=centroid.copy()
                tempCluster=clusterAssment.copy()
        #Update the index
        tempCluster[np.nonzero(tempCluster[:,0]==1)[0],0]=len(centroidList)
        tempCluster[np.nonzero(tempCluster[:,0]==0)[0],0]=maxIndex
        
        #update the information of index and SSE
        clusterInformation[np.nonzero(clusterInformation[:,0]==maxIndex)[0],:]=tempCluster
        #update the centrolist
        centroidList[maxIndex]=tempCentroid[0].tolist()[0]
        centroidList.append(tempCentroid[1].tolist()[0])
    # (List[List[float]], np.matrix)
    return centroidList,clusterInformation

In [18]:
c_list, c_info = bisectingKMeans(X,7,200)

/Users/shriyavanvari/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/shriyavanvari/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [19]:
type(c_list), len(c_list), type(c_list[0]), len(c_list[0]), c_list[0][0]

(list, 7, list, 150, 201.09921568631754)

In [20]:
i=1
abc=""
with open('clustering_v11.0.txt', 'w') as fw:
    print("ItemID,ClusterID",file=fw)
    for v in c_info[:, 0]+1:
        abc=str(i)+","+str(int(v.A[0][0]))
        i=i+1
        print(abc,file=fw)

In [21]:
import pandas as pd

df = pd.read_csv("clustering_v11.0.txt",delimiter=',')
df.to_csv('Submission3.csv',index=False)

In [22]:
# Using silhouette score as the metric
valueK = list()
silhouetteScore = list()
labels = list()
for k in range(3, 22, 2):
    c_list, c_info = bisectingKMeans(X,k,10)
    for v in c_info[:, 0]+1:
        labels.append((int(v.A[0][0])))

    valueK.append(k)
    silhouetteScore.append(metrics.silhouette_score(X, labels))

/Users/shriyavanvari/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/shriyavanvari/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/Users/shriyavanvari/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/shriyavanvari/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


ValueError: Found input variables with inconsistent numbers of samples: [8580, 17160]

In [ ]:
# Plotting silhoutte score
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(valuek, silhouetteScore)
plt.xlabel('Number of Clusters k')
plt.ylabel('Silhouette Score')